# Import Libaries

In [1]:
import numpy as np 

import matplotlib.pyplot as plt


from numpy import *

from sklearn import datasets 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier

import random

from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score


from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor 


from sklearn.metrics import roc_curve, auc

fpath = "c:\\dropbox\\variance\\unsw\\zzsc5836\\raw_data\\"


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

from sklearn.ensemble import RandomForestClassifier


# Functions

In [3]:
def read_data(run_num, prob):

    normalise = False
    
    if prob == 'classifification': #Source:  Pima-Indian diabetes dataset: https://www.kaggle.com/kumargh/pimaindiansdiabetescsv
        data_in = genfromtxt(fpath+"pima.csv", delimiter=",")
        data_inputx = data_in[:,0:8] # all features 0, 1, 2, 3, 4, 5, 6, 7 
        data_inputy = data_in[:,-1] # this is target - so that last col is selected from data

    elif prob == 'regression': # energy - regression prob
        data_in = genfromtxt(fpath+'ENB2012_data.csv', delimiter=",")  
        data_inputx = data_in[:,0:8] # all features 0, - 7
        data_inputy = data_in[:,8] # this is target - just the heating load selected from data
  

    if normalise == True:
        transformer = Normalizer().fit(data_inputx)  # fit does nothing.
        data_inputx = transformer.transform(data_inputx)
 

 
    x_train, x_test, y_train, y_test = train_test_split(data_inputx, data_inputy, test_size=0.40, random_state=run_num)

    return x_train, x_test, y_train, y_test

 
    
def scipy_models(x_train, x_test, y_train, y_test, type_model, hidden, learn_rate, run_num, problem):

    print(run_num, ' is our exp run')

    tree_depth = 2
 
    if problem == 'classifification':
        if type_model ==0: #SGD 
            model = MLPClassifier(hidden_layer_sizes=(hidden,), random_state=run_num, max_iter=100,solver='sgd',  learning_rate_init=learn_rate ) 
        elif type_model ==1: #https://scikit-learn.org/stable/modules/tree.html  (see how tree can be visualised)
            model = DecisionTreeClassifier(random_state=0, max_depth=tree_depth)

    elif problem == 'regression':
        if type_model ==0: #SGD 
            #model = MLPRegressor(hidden_layer_sizes=(hidden,), random_state=run_num, max_iter=100,solver='sgd',  learning_rate_init=learn_rate ) 

            model = MLPRegressor(hidden_layer_sizes=(hidden*3,), random_state=run_num, max_iter=500, solver='adam',learning_rate_init=learn_rate) 
        elif type_model ==1: #https://scikit-learn.org/stable/modules/tree.html  (see how tree can be visualised)
            model = DecisionTreeRegressor(random_state=0, max_depth=tree_depth)
   
    # Train the model using the training sets
    model.fit(x_train, y_train)

    if type_model ==1:
        r = export_text(model)
        # print(r) #print the tree

    # Make predictions using the testing set
    y_pred_test = model.predict(x_test)
    y_pred_train = model.predict(x_train) 

    if problem == 'regression':
        perf_test =  np.sqrt(mean_squared_error(y_test, y_pred_test)) 
        perf_train=  np.sqrt(mean_squared_error(y_train, y_pred_train)) 

    if problem == 'classifification': 
        perf_test = accuracy_score(y_pred_test, y_test) 
        perf_train = accuracy_score(y_pred_train, y_train) 
        cm = confusion_matrix(y_pred_test, y_test) 
        #print(cm, 'is confusion matrix')
        #auc = roc_auc_score(y_pred, y_test, average=None) 

    return perf_test #,perf_train

# Main

In [4]:
max_expruns = 5

SGD_all = np.zeros(max_expruns) 
Adam_all = np.zeros(max_expruns) 
tree_all = np.zeros(max_expruns)  

learn_rate = 0.01
hidden = 8

prob = 'classifification' #  classification  or regression 
#prob = 'regression' #  classification  or regression 


# classifcation accurary is reported for classification and RMSE for regression

print(prob, ' is our problem')


for run_num in range(0,max_expruns): 

    x_train, x_test, y_train, y_test = read_data(run_num, prob)   
    
    acc_sgd = scipy_models(x_train, x_test, y_train, y_test, 0, hidden, learn_rate, run_num, prob) #SGD 
    acc_tree = scipy_models(x_train, x_test, y_train, y_test, 1, hidden, learn_rate,  run_num, prob) #Decision Tree
    
    SGD_all[run_num] = acc_sgd 
    tree_all[run_num] = acc_tree

print(SGD_all,' SGD_all')
print(np.mean(SGD_all), ' mean nn_all')
print(np.std(SGD_all), ' std nn_all')


classifification  is our problem
0  is our exp run
0  is our exp run
1  is our exp run
1  is our exp run
2  is our exp run
2  is our exp run
3  is our exp run
3  is our exp run
4  is our exp run
4  is our exp run
[0.66558442 0.6461039  0.66558442 0.59415584 0.66558442]  SGD_all
0.6474025974025974  mean nn_all
0.02767178669176953  std nn_all


In [5]:
print(tree_all, hidden,' tree_all')

[0.72402597 0.70454545 0.73051948 0.71428571 0.78896104] 8  tree_all


In [6]:
print(np.mean(tree_all),  ' tree _all')
print(np.std(tree_all),  ' tree _all')

0.7324675324675325  tree _all
0.029586456493232507  tree _all


## Random Forest Function

In [7]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
def random_forest(num_trees, num_leaves):

    #rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
    rnd_clf = RandomForestClassifier(n_estimators=num_trees, max_leaf_nodes=num_leaves, n_jobs=-1)
    rnd_clf.fit(x_train, y_train)

    y_pred_rf = rnd_clf.predict(x_test) # This gives us the predicted class for each of the 3 rows of data
    #print(y_pred_rf)

    acc_rf = accuracy_score(y_test, y_pred_rf)

    return acc_rf

In [8]:
# Extra-Trees
from sklearn.ensemble import ExtraTreesClassifier
def extra_trees_func(num_trees, num_leaves):

    #rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
    et_clf = ExtraTreesClassifier(n_estimators=num_trees, max_leaf_nodes=num_leaves, n_jobs=-1)    
    et_clf.fit(x_train, y_train)

    y_pred_rf = et_clf.predict(x_test) # This gives us the predicted class for each of the 3 rows of data
    #print(y_pred_rf)

    acc_et = accuracy_score(y_test, y_pred_rf)

    return acc_et

In [10]:
num_tree_list = [20,40,60]
acc_rf_all = np.zeros(len(num_tree_list))
acc_et_all = np.zeros(len(num_tree_list))

i = 0
for num_trees in num_tree_list:

    acc_rf_all[i] = random_forest(num_trees, 16)
    acc_et_all[i] = extra_trees_func(num_trees, 16)

    i = i+1

print(f"Accuracy Scores for Random Forests: {acc_rf_all}, Mean: {np.mean(acc_rf_all)}")
print(f"Accuracy Scores for Extra Trees: {acc_et_all}, Mean: {np.mean(acc_et_all)}")


Accuracy Scores for Random Forests: [0.78246753 0.80844156 0.79545455], Mean: 0.7954545454545454
Accuracy Scores for Extra Trees: [0.79545455 0.79220779 0.78246753], Mean: 0.79004329004329
